## Reading, writing and displaying depth file data

Note that a lot of function assume a square grid of with equal x and y gridsteps! Also assumes grid width and length are even (cleanly divisible by 2), so watch it.

In [1]:
%matplotlib widget

In [2]:
from JulesD3D.DepthModel import DepthModel
from JulesD3D.utils import quickDF
from os import path, mkdir

In [3]:
%load_ext autoreload

In [4]:
%reload_ext autoreload

In [5]:
# Better to give channel width an int of x gridsteps width!
# TODO improvement: Define whole channel dimensions in gridsteps!
channel = {
    "width": 400,    # [m]  has to be at least x gridstep wide
    "slope": 1.25,   # [deg]
    "length": 15000, # [m] has to be multiple of y gridstep
    "depth": 150,     # [m] 
#     "bank_depth": 100  # [m] Ignored if channel depth is given
}

grid = {
    "length": 36000,      # y [m] has to be multiple of y gridstep
    "width":  26000,       # x [m] has to be multiple of x gridstep
    "x_gridstep": 200,    # [m]
    "y_gridstep": 200,    # [m]
    "dims": [],
}


bathymetry = {
    "initial_depth": 150, # [m]
    "slope": 0.2,         # [m] the 'basin' slope
}

folder = '/Users/julesblom/ThesisPython/generated'

width_in_km_str = str(int(grid['width']/1000))
length_in_km_str = str(int(grid['length']/1000))
grid_str = str(grid['x_gridstep'])

dep_filename = f"gen_{length_in_km_str}_by_{width_in_km_str}km.dep"
grid_filename = f"{length_in_km_str}km_{width_in_km_str}km_W60Channel.grd"
enc_filename = f"{length_in_km_str}km_1cellchannel_15km.enc"

filenames = {
    "grid": path.join(folder, grid_filename),
    "dep": path.join(folder, dep_filename),
    "enc": path.join(folder, enc_filename)
}

# Remember to change MNKMax in .mdf and location in .bnd files

In [6]:
new_model = DepthModel(filenames=filenames, channel=channel, bathymetry=bathymetry, grid=grid)

In [7]:
new_model.makeModelAtOnce()

------ Making new Delft3D grid ------
x_gridstep 200
y_gridstep 200
width 26000
length 36000
MNKmax = 132 182 SIGMALAYERS
xDim 131
yDim 181
Writing grid file with shape (131, 181)
------ Making bathymetry ------
* Minimum depth:  150
* Channel slope:  1.25
* Basin slope: 	 0.2
channel_length_index 76
channel_grid_cells 3
* bank_width 12800.0
* bank_index 64
bank_left 64
bank_right 67

depth_matrix[bank_left:bank_right,:] shape (3, 182)
channel (3, 182)
--- Writing enclosure file ---
xDim:  131
yDim:  181
 ----- Writing enclosure file with channel -----
Writing depth file to: /Users/julesblom/ThesisPython/generated/gen_36_by_26km.dep
Writing .dep file with shape (182, 132)


## Easy copy new boundary conditions

In [8]:
dims = new_model.grid['dims']

In [9]:
xDim, yDim = dims[0]
xDim, yDim

In [11]:
print(f"MNKmax = {xDim+1} {yDim+1} 80")

MNKmax = 132 182 80


In [12]:
discharge_location = int((xDim-1)/2+1)

new_boundary_file = f'''Discharge            Q T    {discharge_location}     1    {discharge_location}     1  2.0000000e+002 3d-profile
deep                 Z T     2   {yDim+1}   {xDim}   {yDim+1}  2.0000000e+002'''
print(new_boundary_file)

Discharge            Q T    66     1    66     1  2.0000000e+002 3d-profile
deep                 Z T     2   182   131   182  2.0000000e+002


In [13]:
new_model.plot3D()

x shape: 131
y shape: 181
z shape: (181, 131)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
new_model.plotCrossSection()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# quickDF(new_model.bathymetry['depth'])

In [16]:
new_model.plotGrid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# new_model.grid['width']/300/2+1
# 18000/300/2+1
# 20100/300
70*300

21000

In [18]:
new_model.plotDepthAndGrid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<hr/>

### Generate bathymetries with different slopes

In [19]:
# channel_slopes = [1.00, 1.25, 1.5, 1.75] # [degrees]

In [20]:
# for channel_slope in channel_slopes:
#     print("---------- Making depth model with slope:", channel_slope, '----------')

#     # modify slope in channel object defined above!
#     channel['slope'] = channel_slope

# #     bathymetry['slope'] = basin_slope
    
#     new_folder = '/Users/julesblom/ThesisPython/generated/Slope{}/'.format(str(channel_slope))

#     try:  
#         print("Making new folder:", new_folder)
#         mkdir(new_folder)
#     except OSError:
#         dir(OSError)
#         print("Creation of the directory failed, it probably already exists!")
    
#     new_filenames = {
#         "grid": path.join(new_folder, '45km_300m_W60Channel.grd'),
#         "dep": path.join(new_folder, 'gen_18_by_45km.dep'), # folder_name + '/slopeBreakSmooth.dep' #
#         "enc": path.join(new_folder, '45km_1cellchannel_15km.enc')
#     }
    
#     new_slope_model = DepthModel(filenames=new_filenames, channel=channel, bathymetry=bathymetry, grid=grid)
#     new_slope_model.makeModelAtOnce()

# PyVista 3D generated bathymetry

In [21]:
import pyvista as pv
# from JulesD3D.plotPyVista import makeBottomSurface
from cmocean.cm import deep_r
from JulesD3D.enc import readEnc
import pandas as pd
from numpy import ones

In [22]:
# enc_filename = '/Users/julesblom/ThesisPython/generated/45km_1cellchannel_15km.enc'
# enclosure_x, enclosure_y = readEnc(enc_filename)
# enclosure_z = ones(len(enclosure_x))
# plot_enclosure = zip(enclosure_x, enclosure_y, enclosure_z)

In [23]:
depth = new_model.bathymetry['depth'][1:-1,1:-1]

plot_x_mesh = new_model.grid['x_grid'][:-1,:-1]
plot_y_mesh = new_model.grid['y_grid'][:-1,:-1]
plot_z_mesh = -depth

bottom_surface = pv.StructuredGrid(plot_x_mesh, plot_y_mesh, plot_z_mesh)
bottom_surface.field_arrays['depth'] = -depth.T
bottom_surface

StructuredGrid (0x129ff78a0)
  N Cells:	23091
  N Points:	23400
  X Bounds:	0.000e+00, 2.580e+04
  Y Bounds:	1.000e+02, 3.590e+04
  Z Bounds:	-6.990e+02, -1.540e+02
  Dimensions:	180, 130, 1
  N Arrays:	1

In [ ]:
p = pv.Plotter(notebook=False)
p.add_mesh(bottom_surface, cmap=deep_r, scalars='depth', ambient=0.2)
# p.add_lines(plot_enclosure)
p.show_grid()
p.set_scale(zscale=10)
p.show()

In [ ]:
p.close()